### Import the libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [94]:
# import the data
train=pd.read_csv('train (1).csv')
test=pd.read_csv('test (1).csv')

In [95]:
# Cleaning the data and making it prepare for next step
print(train.shape,test.shape)

(1460, 81) (1459, 80)


In [96]:
cor=train.corr()
cor=pd.DataFrame(cor['SalePrice'])
cor=cor[(cor['SalePrice']<=0.2) & (cor['SalePrice']>=-0.2)]
cor=cor.T
cor.columns

Index(['Id', 'MSSubClass', 'OverallCond', 'BsmtFinSF2', 'LowQualFinSF',
       'BsmtHalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'EnclosedPorch',
       '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold'],
      dtype='object')

In [97]:
for i in cor.columns:
    train.drop(i,axis=1,inplace=True)

In [98]:
train.head()

,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,PoolQC,Fence,MiscFeature,SaleType,SaleCondition,SalePrice
0,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,...,TA,Y,0,61,NaN,NaN,NaN,WD,Normal,208500
1,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,...,TA,Y,298,0,NaN,NaN,NaN,WD,Normal,181500
2,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,...,TA,Y,0,42,NaN,NaN,NaN,WD,Normal,223500
3,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,...,TA,Y,0,35,NaN,NaN,NaN,WD,Abnorml,140000
4,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,...,TA,Y,192,84,NaN,NaN,NaN,WD,Normal,250000


In [99]:

# drop the columns which has more than 600 null values
for i in train.columns:
    if train[i].isnull().sum()>600:
        train.drop(i,axis=1,inplace=True)

In [100]:
train.head()

,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,SaleType,SaleCondition,SalePrice
0,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,...,2,548,TA,TA,Y,0,61,WD,Normal,208500
1,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,...,2,460,TA,TA,Y,298,0,WD,Normal,181500
2,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,...,2,608,TA,TA,Y,0,42,WD,Normal,223500
3,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,...,3,642,TA,TA,Y,0,35,WD,Abnorml,140000
4,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,...,3,836,TA,TA,Y,192,84,WD,Normal,250000


In [101]:
def iqmean(data):
    q3, q1 = np.percentile(data, [75 ,25])
    iqr=q3-q1
    iqrange=data[(data > q1) & (data < q3)]
    mean=sum(iqrange)/(data.shape[0]/2)
    return mean


In [102]:
for i in (train.columns):
    if (train[i].dtypes=='int64') | (train[i].dtypes=='float64'):
        train[i].fillna(iqmean(train[i]),inplace=True)

In [103]:
for i in (train.columns):
    if train[i].dtypes=='O':
        train[i].fillna(str(train[i].mode()),inplace=True)

train.dropna(axis=0,inplace=True)
train.shape

In [104]:
train.isnull().sum().sum()

0

# Split the data to train and test


In [105]:
x=train.drop('SalePrice',axis=1)
y=train.SalePrice

In [106]:
# change the nominal data to ordinal
x=pd.get_dummies(x)

In [107]:
# Now import the algorithm for splitting the data
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25)

In [108]:
y_test

989     197000
996     136500
840     140000
940     150900
608     359100
         ...  
908     131000
938     239799
1125    115000
707     254000
1394    246578
Name: SalePrice, Length: 365, dtype: int64

In [109]:
from sklearn.linear_model import LinearRegression

In [110]:
Lmodel=LinearRegression()

In [111]:
Lmodel.fit(x_train,y_train)

LinearRegression()

In [112]:
pred=Lmodel.predict(x_test)

In [113]:
# import mean square metrics for checking the accuracy
from sklearn.metrics import mean_squared_error

In [114]:
mean_squared_error(pred,y_test)

838700770.4057353

In [115]:
# We cannot get a better idea from mean squared error so it is better to check with the correlation between the predicted and 
# observed data

In [116]:
a={'predicted':pred,'observed':y_test}

In [117]:
a=pd.DataFrame(a)
a.corr()

,predicted,observed
predicted,1.000000,0.930739
observed,0.930739,1.000000


# Decision Tree Classifier

In [154]:
from sklearn.tree import DecisionTreeClassifier
modeltree=DecisionTreeClassifier(criterion='gini',max_depth=24,)

In [155]:
modeltree.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=24)

In [156]:
ppp=modeltree.predict(x_test)

In [157]:
mean_squared_error(ppp,y_test)

2497126866.528767

In [158]:
a={'predicted':ppp,'observed':y_test}
a=pd.DataFrame(a)
a.corr()

,predicted,observed
predicted,1.000000,0.776995
observed,0.776995,1.000000


# now import the testing data which the prediction should be made on 

In [542]:
test=pd.read_csv('test (1).csv')

In [543]:
test.shape

(1459, 80)

In [544]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [545]:
#drop the columns which we already droped from the training data 
for i in test.columns:
    if i not in train.columns:
        test.drop(i,axis=1,inplace=True)

In [546]:
test.shape

(1459, 60)

In [547]:
train.shape

(1460, 61)

In [548]:
 #Fill the nan values of testfinal table
for i in (test.columns):
    if (test[i].dtypes=='int64') | (test[i].dtypes=='float64'):
        test[i].fillna(iqmean(test[i]),inplace=True)

In [549]:
for i in (test.columns):
    if test[i].dtypes=='O':
        test[i].fillna(str(test[i].mode()),inplace=True)

In [550]:
testfinal=pd.get_dummies(test)
testfinal.shape

(1459, 257)

In [551]:
x.shape

(1460, 249)

In [552]:
#There are 27 features more in training data than the testing data 
for i in x.columns:
    if i not in testfinal.columns:
        x.drop(i,axis=1,inplace=True)

In [553]:
for i in testfinal.columns:
    if i not in x.columns:
        testfinal.drop(i,axis=1,inplace=True)

In [554]:
testfinal.shape

(1459, 249)

In [555]:
x.shape

(1460, 249)

In [556]:
testfinal.isnull().sum().sum()

0

# again train the model with the whole training data and perform prediction based on that

In [557]:
Linmodel=LinearRegression()
Linmodel.fit(x,y)
prediction=Linmodel.predict(testfinal)
prediction

array([102663.36120014, 152723.88021221, 180512.43646087, ...,
       161591.84015006, 343428.76692216, 218759.6196483 ])

In [558]:
prediction

array([102663.36120014, 152723.88021221, 180512.43646087, ...,
       161591.84015006, 343428.76692216, 218759.6196483 ])

In [559]:
test2=pd.read_csv('test (1).csv')
test2['SalePrice']=prediction

In [560]:
test2.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,102663.361200
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,152723.880212
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,180512.436461
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2010,WD,Normal,186784.605129
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,0,NaN,NaN,NaN,0,1,2010,WD,Normal,204820.099924


In [561]:
file=test2.loc[:,['Id','SalePrice']]
file

,Id,SalePrice
0,1461,102663.361200
1,1462,152723.880212
2,1463,180512.436461
3,1464,186784.605129
4,1465,204820.099924
...,...,...
1454,2915,308571.070721
1455,2916,83681.274749
1456,2917,161591.840150
1457,2918,343428.766922


In [562]:
file.to_csv('Predition_houseprice3.csv')

In [ ]:
# A try with pol

In [563]:
from sklearn.preprocessing import PolynomialFeatures
polmodel=PolynomialFeatures(degree=2)

In [564]:
xpoly=polmodel.fit_transform(x_train)
polmodel.fit(x_train,y_train)

PolynomialFeatures()

In [565]:
Linmodel.fit(xpoly,y_train)

LinearRegression()

In [566]:
pred=Linmodel.predict(polmodel.fit_transform(x_test))

In [567]:
mean_squared_error(pred,y_test)

19820926612.822197

In [568]:
from sklearn.neighbors import KNeighborsClassifier

In [569]:
knnmod=KNeighborsClassifier(n_neighbors=41)
knnmod.fit(x_train,y_train)

KNeighborsClassifier(n_neighbors=41)

In [570]:
predd=knnmod.predict(x_test)

In [571]:
mean_squared_error(predd,y_test)

5234688644.391781

In [572]:
from scipy.stats import zscore

In [311]:
x=x.apply(zscore)
x.isnull().sum().sum()

0

In [312]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25)

In [313]:
Linmodel.fit(x_train,y_train)

LinearRegression()

In [314]:
preddd=Linmodel.predict(x_test)

In [315]:
mean_squared_error(preddd,y_test)

8.093308144769437e+29